In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from scipy.stats import expon, reciprocal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras



In [38]:
# Load the dataset
train_df = pd.read_csv('csv_files/train.csv')

# Convert binary categorical features to 0 and 1
binary_features = ['CryoSleep', 'VIP']
train_df[binary_features] = train_df[binary_features].astype(bool).astype(int)

# Extract components from 'Cabin' and treat them as separate features
train_df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = train_df['Cabin'].str.split('/', expand=True)
train_df['Cabin_Side'] = train_df['Cabin_Side'].map({'P': 1, 'S': 0})

# Explicitly convert 'Cabin_Number' to numeric, handling errors
train_df['Cabin_Number'] = pd.to_numeric(train_df['Cabin_Number'], errors='coerce')

# Drop the original 'Cabin' column as it's no longer needed
train_df.drop('Cabin', axis=1, inplace=True)

# One-hot encode multi-category features
multi_cat_features = ['HomePlanet', 'Destination']
one_hot_encoder = OneHotEncoder(drop='first')  # Adjusted here
encoded_features = one_hot_encoder.fit_transform(train_df[multi_cat_features])
encoded_feature_names = one_hot_encoder.get_feature_names_out(multi_cat_features)

# Convert to DataFrame and concatenate
encoded_features_df = pd.DataFrame(encoded_features.toarray(), columns=encoded_feature_names)

# Drop the original columns and concatenate the new one-hot encoded features
train_df.drop(multi_cat_features, axis=1, inplace=True)
train_df = pd.concat([train_df.reset_index(drop=True), encoded_features_df], axis=1)

# Convert 'Transported' to integer (True=1, False=0) for modeling
train_df['Transported'] = train_df['Transported'].astype(int)

# Define numeric features - ensure this list only contains numeric features that were not one-hot encoded
numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin_Number', 'Cabin_Side']
# Imputer for numeric features
imputer = SimpleImputer(strategy='median')
train_df[numeric_features] = imputer.fit_transform(train_df[numeric_features])

# Normalize the numeric features
scaler = StandardScaler()
train_df[numeric_features] = scaler.fit_transform(train_df[numeric_features])

# Prepare features and target for the model
features = [col for col in train_df.columns if col not in ['PassengerId', 'Name', 'Transported', 'Cabin_Deck' ]]  # Exclude non-features
X = train_df[features]
y = train_df['Transported']

# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Building the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Evaluate on the validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=1)
print(f'Validation Accuracy: {val_acc}')


Epoch 1/50


/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.6912 - loss: 0.5818 - val_accuracy: 0.7717 - val_loss: 0.4668
Epoch 2/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.7850 - loss: 0.4513 - val_accuracy: 0.7832 - val_loss: 0.4467
Epoch 3/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - accuracy: 0.7885 - loss: 0.4479 - val_accuracy: 0.7855 - val_loss: 0.4423
Epoch 4/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step - accuracy: 0.7970 - loss: 0.4383 - val_accuracy: 0.7815 - val_loss: 0.4470
Epoch 5/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - accuracy: 0.7930 - loss: 0.4274 - val_accuracy: 0.7872 - val_loss: 0.4427
Epoch 6/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.7898 - loss: 0.4402 - val_accuracy: 0.7803 - val_loss: 0.4418
Epoch 7/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.8034 - loss: 0.4159 - val_accuracy: 0.7832 - val_loss: 0.4351
Epoch 8/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step - accuracy: 0.8045 - loss: 0.4200 - val_accurac

In [40]:
# Load the test dataset
test_df = pd.read_csv('csv_files/test.csv')

# Convert binary categorical features to 0 and 1 for the test data
test_df[binary_features] = test_df[binary_features].astype(bool).astype(int)

# Extract components from 'Cabin' and treat them as separate features for the test data
test_df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = test_df['Cabin'].str.split('/', expand=True)
test_df['Cabin_Side'] = test_df['Cabin_Side'].map({'P': 1, 'S': 0})
test_df['Cabin_Number'] = pd.to_numeric(test_df['Cabin_Number'], errors='coerce')

# Drop the original 'Cabin' column as it's no longer needed for the test data
test_df.drop('Cabin', axis=1, inplace=True)

# One-hot encode multi-category features for the test data using the same encoder
encoded_features_test = one_hot_encoder.transform(test_df[multi_cat_features])
encoded_features_test_df = pd.DataFrame(encoded_features_test.toarray(), columns=encoded_feature_names)

# Drop the original multi-category columns and concatenate the new one-hot encoded features for the test data
test_df.drop(multi_cat_features, axis=1, inplace=True)
test_df = pd.concat([test_df.reset_index(drop=True), encoded_features_test_df.reset_index(drop=True)], axis=1)

# Impute missing values for numeric features only in the test data
test_df[numeric_features] = imputer.transform(test_df[numeric_features])

# Normalize the numeric features in the test data
test_df[numeric_features] = scaler.transform(test_df[numeric_features])

# Prepare the features for prediction
X_test = test_df[features]

# Make predictions with the TensorFlow model
y_pred_test_proba = model.predict(X_test)
y_pred_test = (y_pred_test_proba > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_pred_test
})

# Convert predictions back to boolean (True/False) if necessary
submission_df['Transported'] = submission_df['Transported'].astype(bool)

# Save the submission file
submission_df.to_csv('tensorflow_result5.csv', index=False)


134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step
